In [53]:
import feedparser
from afinn import Afinn
import pandas as pd

In [59]:
feed = feedparser.parse("http://politiken.dk/rss/senestenyt.rss")
afinn = Afinn (language = "da")
data_titles = []
data_scores = []
for i in range(len(feed["entries"])):
    data_titles.append(feed["entries"][i]["title"])
    data_scores.append(afinn.score(feed["entries"][i]["title"]))

pd.set_option('display.max_colwidth', -1)
pd.DataFrame({"Title": data_titles, "Score": data_scores})

,Title,Score
0,"Optog, mindekoncerter og røverhistorier: Her kan du være med til hylde Kim Larsen",2.0
1,"Kommentar: Det er de tilbageværende medlemmer af Det Svenske Akademi, der bagatelliserede sexovergreb",0.0
2,"Nekrolog: Han blev en kunstner, man græd og lo sammen med",1.0
3,En weekend i: Madrid er et festfyrværkeri af spændende kunst,3.0
4,1320% mere streaming: Kim Larsen stormer frem på danske musiktjenester,0.0
5,"Daniel Kunne ikke få adgang til sine egne sundhedsdata: Nu udvikler han et elektronisk plaster, der kan måle dine biometiske data",0.0
6,Noa Redington: Partierne har endnu ikke lagt sig fast på de endelige budskaber. Det skal de være glade for. Den kreative metaltræthed er udtalt,3.0
7,SF boykotter Folketingets åbningsgudstjeneste i protest mod præst,-2.0
8,"Kim Larsens søn: »Nej, han var langt fra perfekt, den gamle. Men hvor jeg dog elskede ham«",5.0
9,Efter kritik og opsigelse: Nu får kunstakademiet ny rektor,-2.0
